# Ingestão de Dados - PostgreSQL

Este notebook realiza a ingestão de dados de um banco de dados PostgreSQL para o MinIO usando DeltaLake.

## Configuração

Configure as variáveis abaixo antes de executar:

In [ ]:
# Importar configurações base
%run ../00_configuracao_inicial.ipynb

In [ ]:
# ============================================
# CONFIGURAÇÕES DE CONEXÃO POSTGRESQL
# ============================================
import os

# Configurações de conexão PostgreSQL
POSTGRES_HOST = os.getenv('POSTGRES_HOST', 'localhost')
POSTGRES_PORT = os.getenv('POSTGRES_PORT', '5432')
POSTGRES_DATABASE = os.getenv('POSTGRES_DATABASE', 'postgres')
POSTGRES_USER = os.getenv('POSTGRES_USER', 'postgres')
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD', 'senha')

# Configurações de leitura
POSTGRES_SCHEMA = os.getenv('POSTGRES_SCHEMA', 'public')
POSTGRES_TABLE = os.getenv('POSTGRES_TABLE', 'nome_tabela')

# Configurações de destino no MinIO
DESTINO_BRONZE = f"{PATH_BRONZE}/postgresql/{POSTGRES_DATABASE.lower()}/{POSTGRES_SCHEMA.lower()}/{POSTGRES_TABLE.lower()}"

print("Configurações PostgreSQL:")
print(f"Host: {POSTGRES_HOST}")
print(f"Port: {POSTGRES_PORT}")
print(f"Database: {POSTGRES_DATABASE}")
print(f"Schema: {POSTGRES_SCHEMA}")
print(f"Table: {POSTGRES_TABLE}")
print(f"Destino: {DESTINO_BRONZE}")

In [ ]:
# Instalar driver PostgreSQL (executar apenas uma vez)
# !pip install psycopg2-binary

# O Spark já inclui o driver JDBC PostgreSQL por padrão

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# URL de conexão JDBC PostgreSQL
jdbc_url = f"jdbc:postgresql://{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DATABASE}"

# Propriedades de conexão
connection_properties = {
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
    "driver": "org.postgresql.Driver"
}

print(f"JDBC URL: {jdbc_url}")

In [ ]:
# Função para ler dados do PostgreSQL
def ler_postgresql_table(table_name, schema="public", query=None, partition_column=None, num_partitions=None, lower_bound=None, upper_bound=None):
    """
    Lê dados de uma tabela PostgreSQL
    
    Args:
        table_name: Nome da tabela
        schema: Schema (padrão: public)
        query: Query SQL customizada (opcional, substitui table_name)
        partition_column: Coluna para particionamento paralelo (opcional)
        num_partitions: Número de partições (opcional)
        lower_bound: Valor mínimo para particionamento (opcional)
        upper_bound: Valor máximo para particionamento (opcional)
    
    Returns:
        DataFrame do Spark
    """
    if query:
        # Usar query customizada (subquery)
        table_or_query = f"({query}) postgres_table"
    elif schema:
        table_or_query = f"{schema}.{table_name}"
    else:
        table_or_query = table_name
    
    reader = spark.read.format("jdbc") \
        .option("url", jdbc_url) \
        .option("dbtable", table_or_query) \
        .option("user", POSTGRES_USER) \
        .option("password", POSTGRES_PASSWORD) \
        .option("driver", "org.postgresql.Driver")
    
    # Adicionar opções de particionamento se fornecidas
    if partition_column and num_partitions:
        reader = reader.option("partitionColumn", partition_column) \
                      .option("numPartitions", num_partitions)
        if lower_bound is not None and upper_bound is not None:
            reader = reader.option("lowerBound", lower_bound) \
                          .option("upperBound", upper_bound)
    
    df = reader.load()
    
    return df

In [ ]:
# Exemplo 1: Leitura simples de tabela
print("Exemplo 1: Leitura simples")
df_postgres = ler_postgresql_table(
    table_name=POSTGRES_TABLE,
    schema=POSTGRES_SCHEMA
)

print(f"Total de registros: {df_postgres.count()}")
df_postgres.printSchema()
df_postgres.show(5, truncate=False)

In [ ]:
# Exemplo 2: Leitura com query customizada
print("Exemplo 2: Leitura com query customizada")
query_customizada = f"""
    SELECT 
        coluna1,
        coluna2,
        coluna3,
        updated_at
    FROM {POSTGRES_SCHEMA}.{POSTGRES_TABLE}
    WHERE updated_at >= NOW() - INTERVAL '30 days'
    ORDER BY updated_at DESC
"""

# df_postgres_query = ler_postgresql_table(query=query_customizada)
# df_postgres_query.show(5)

In [ ]:
# Exemplo 3: Leitura com particionamento paralelo (para tabelas grandes)
print("Exemplo 3: Leitura com particionamento")
# df_postgres_partitioned = ler_postgresql_table(
#     table_name=POSTGRES_TABLE,
#     schema=POSTGRES_SCHEMA,
#     partition_column="id",  # Coluna numérica para particionamento
#     num_partitions=10,
#     lower_bound=1,
#     upper_bound=1000000
# )
# df_postgres_partitioned.show(5)

In [ ]:
# Adicionar metadados de ingestão
df_ingestao = df_postgres \
    .withColumn("fonte", lit("POSTGRESQL")) \
    .withColumn("database_origem", lit(POSTGRES_DATABASE)) \
    .withColumn("schema_origem", lit(POSTGRES_SCHEMA)) \
    .withColumn("tabela_origem", lit(POSTGRES_TABLE)) \
    .withColumn("ingestao_em", current_timestamp()) \
    .withColumn("particao_data", date_format(current_date(), "yyyy-MM-dd"))

print("Metadados adicionados:")
df_ingestao.select("fonte", "database_origem", "schema_origem", "tabela_origem", "ingestao_em").show(1, truncate=False)

In [ ]:
# Salvar no MinIO como Delta Table
print(f"Salvando dados em: {DESTINO_BRONZE}")

# save_delta_table(
#     df_ingestao,
#     DESTINO_BRONZE,
#     mode="overwrite",  # ou "append" para incrementais
#     partition_by=["particao_data"]  # Particionar por data
# )

print("Ingestão concluída com sucesso!")

In [ ]:
# Verificar dados salvos
# df_verificacao = read_delta_table(DESTINO_BRONZE)
# print(f"Registros salvos: {df_verificacao.count()}")
# df_verificacao.show(5)

## Ingestão Incremental

Para ingestões incrementais baseadas em timestamp ou ID:

In [ ]:
# Função para ingestão incremental
def ingestao_incremental_postgresql(table_name, schema, coluna_timestamp="updated_at", ultima_execucao=None):
    """
    Realiza ingestão incremental de dados PostgreSQL
    
    Args:
        table_name: Nome da tabela
        schema: Schema
        coluna_timestamp: Nome da coluna de timestamp para filtro
        ultima_execucao: Timestamp da última execução (formato: 'YYYY-MM-DD HH24:MI:SS')
    """
    if ultima_execucao:
        query = f"""
            SELECT * FROM {schema}.{table_name}
            WHERE {coluna_timestamp} > '{ultima_execucao}'::timestamp
            ORDER BY {coluna_timestamp}
        """
    else:
        # Primeira execução: pegar últimos 7 dias
        query = f"""
            SELECT * FROM {schema}.{table_name}
            WHERE {coluna_timestamp} >= NOW() - INTERVAL '7 days'
            ORDER BY {coluna_timestamp}
        """
    
    df_incremental = ler_postgresql_table(query=query)
    
    return df_incremental

# Exemplo de uso
# df_incremental = ingestao_incremental_postgresql(
#     table_name=POSTGRES_TABLE,
#     schema=POSTGRES_SCHEMA,
#     ultima_execucao="2024-01-01 00:00:00"
# )
# 
# # Salvar em modo append
# save_delta_table(df_incremental, DESTINO_BRONZE, mode="append", partition_by=["particao_data"])